In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import sys
import os 

os.chdir("../")

from einops import rearrange
import einops 
import numpy as np
import torch

from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from typing import List
from torchvision.transforms import Compose, ToTensor, Lambda

device = "cuda" if torch.cuda.is_available() else "cpu"


# Datasets

In [40]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
dataset_train = dataset["train"].map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
dataset_train.set_format(type='torch', columns=['input_ids', 'text', 'attention_mask'])
dataset_train.save_to_disk("datasets/wikitext")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /home/johannes/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
from datasets import load_dataset
dataset = load_dataset("wikipedia", "20220301.en")

Found cached dataset wikipedia (/home/johannes/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [111]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
dataset_train = dataset["train"].map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
dataset_train.set_format(type='torch', columns=['input_ids', 'text', 'attention_mask'])
dataset_train.save_to_disk("datasets/wikitext_xlmr")

Found cached dataset wikitext (/home/johannes/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1802 [00:00<?, ?ba/s]

In [3]:
from datasets import load_from_disk
reloaded_dataset = load_from_disk("datasets/wikitext_xlmr")
dataloader = torch.utils.data.DataLoader(reloaded_dataset, batch_size=5)

In [4]:
i = 0
for batch in dataloader:
    
    i +=1
    if i > 5:
        break

In [ ]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer([text, text], return_tensors='pt')
output = model(**encoded_input)

In [17]:
from transformer_model import TransformerModel

decoder = torch.nn.Sequential(
    xlmr_model.lm_head
)

transformer_model = TransformerModel(
    nlayers=8,
    decoder=decoder,
    timesteps=TIMESTEPS,
    hidden_dim=1024,
    max_seq_len=512,
).to(device)

In [6]:
input_ids = batch["input_ids"].to(device)
attention_mask = batch["attention_mask"].to(device)

with torch.no_grad():
    
    xlmr_model.eval()

    output = xlmr_model.forward(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True,
    ).hidden_states[-1]

In [39]:
with torch.no_grad():
    
    xt, noise, t = diffusion_linear.forward(output)

    # add noise, Analyze the distribution for the hidden state
    y, y_decoded = transformer_model(xt, attention_mask=attention_mask, t=t)
    
    pos = transformer_model._pos_encoder(output)
    
    

In [32]:
yt = transformer_model._time_embedder(t).unsqueeze(1)

In [33]:
cc = yt + y

In [35]:
cc = cc - yt

In [37]:
(cc - y).float().mean()

tensor(1.0732e-10, device='cuda:0', grad_fn=<MeanBackward0>)

In [24]:
t

torch.Size([5])

In [19]:
from diffusion import Diffusion

diffusion_linear = Diffusion(
    timesteps=TIMESTEPS,
    schedular="linear",
    start=0.0001,
    end=0.002
)

In [15]:
input_ids = batch["input_ids"].to(device)
attention_mask = batch["attention_mask"].to(device)

xlmr_model.eval()

import time 

s = time.time()

with torch.no_grad():

    output = xlmr_model.forward(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True,
    )

    output_hidden = output.hidden_states[-1]

e = time.time()
  
e - s

0.011213302612304688

In [16]:
t = 500

with torch.no_grad():
    decoder.eval()
    xt, noise, t = diffusion_linear.forward(output_hidden, torch.Tensor([t] * output_hidden.shape[0]).long().to(device))
    
    output_decoded = decoder(xt)
    
    y = transformer_model(xt, attention_mask)

In [ ]:
from diffusion import Diffusion

def process_backward(
    model: torch.nn.Module,
    diffusion: Diffusion,
    timesteps: int,
    shape: List[int],
    device: str,
    x: torch.Tensor = None,
):
    
    model.eval()
    
    with torch.no_grad():

        x = torch.randn(shape, device=device) if x is None else x
        timesteps_iter = list(range(timesteps))
        timesteps_iter.reverse()
        
        attention_mask = torch.ones([1, MAX_SEQUENCE_LENGTH]).to(device)
        
        Y = [x]
        
        for t in timesteps_iter:

            t = torch.Tensor([t]).long().to(device)

            predicted_noise, _ = model.forward(
                x=x,
                t=t,
                attention_mask=attention_mask,
            #    klass
            )
            
            if torch.isnan(predicted_noise).sum() > 0:
                print(t)
            
            x = diffusion.backward(x, predicted_noise, t)
            
            Y.append(x)
            
        return torch.cat(Y, dim=0)

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

TIMESTEPS = 300
MAX_SEQUENCE_LENGTH = 512
HIDDEN_DIM = 1024
BATCH_SIZE = 5

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
xlmr_model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large").to(device)

from datasets import load_from_disk
reloaded_dataset = load_from_disk("datasets/wikitext_xlmr")
dataloader = torch.utils.data.DataLoader(reloaded_dataset, batch_size=BATCH_SIZE)

from transformer_model import TransformerModel

decoder = torch.nn.Sequential(
    xlmr_model.lm_head
)

transformer_model = TransformerModel(
    nlayers=8,
    decoder=None,
    timesteps=TIMESTEPS,
    hidden_dim=HIDDEN_DIM,
    max_seq_len=MAX_SEQUENCE_LENGTH,
).to(device)

In [3]:
from torch.utils.tensorboard import SummaryWriter
from diffusion import Diffusion

diffusion_linear = Diffusion(
    timesteps=TIMESTEPS,
    schedular="linear",
    start=0.0001,
    end=0.002
)

model = transformer_model
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

iteration = 0
image_saves = 0

In [6]:
with torch.no_grad():
    output_hidden = xlmr_model.forward(
        input_ids=input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True,
    ).hidden_states[-1]


In [8]:
iteration = 299

In [ ]:
EXPERIMENT_NAME = "test_transformers"

writer = SummaryWriter(f"runs/{EXPERIMENT_NAME}")

for e in range(1):

    for step, batch in enumerate(dataloader):
        
        iteration += 1
        
        model.train()
        xlmr_model.eval()
        decoder.eval()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        with torch.no_grad():
            
            output_hidden = xlmr_model.forward(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True,
            ).hidden_states[-1]

            x_noised, noise, time = diffusion_linear.forward(output_hidden)
        
        noise_predicted, _ = model.forward(
            x=x_noised,
            t=time,
            attention_mask=attention_mask
        )

        loss = diffusion_linear.loss(noise, noise_predicted)
        
        model.zero_grad()
        loss.backward()
        
        # torch.nn.utils.clip_grad_norm(parameters=model.parameters(), max_norm=2, norm_type=2)

        optimizer.step()
        
        writer.add_scalar("loss", loss, iteration)
        
        if iteration % 300 == 0:
            
            with torch.no_grad():

                y_final = process_backward(
                    model=model,
                    diffusion=diffusion_linear,
                    timesteps=TIMESTEPS,
                    shape=[1, MAX_SEQUENCE_LENGTH, HIDDEN_DIM],
                    device=device,
                )[-1, :, :]

                decoder.eval()
                output_decoded = decoder(y_final)

                output_max = list(output_decoded.argmax(1).detach().cpu().numpy())

                text = tokenizer.decode(output_max)

                writer.add_text("model_output", text, global_step=image_saves)

            image_saves += 1
            
            model.save(folder=f"ckp/{EXPERIMENT_NAME}", name="best.ckp")

    print(e)

In [9]:
time

tensor([370, 638, 503, 956, 946], device='cuda:0')

torch.Size([5, 512, 1024])

In [20]:
with torch.no_grad():
    
    y_final = process_backward(
        model=model,
        diffusion=diffusion_linear,
        timesteps=TIMESTEPS,
        shape=[1, MAX_SEQUENCE_LENGTH, HIDDEN_DIM],
        device=device,
    )[-1, :, :]

    decoder.eval()
    output_decoded = model._decoder(y_final)
    
    output_max = list(output_decoded.argmax(1).detach().cpu().numpy())

    text = tokenizer.decode(output_max)
    
    writer.add_text("first_text", text, global_step=0)
